# Halfar Dome Model

Replicating Luke Morris and Jadon Clugston's notebooks
* https://www.cise.ufl.edu/~luke.morris/cism.html
* https://github.com/JuliaComputing/ASKEMDemos/blob/main/Glacial%20Flow/GlacialFlowNotebook.jl

In [321]:
using Catlab
using Catlab.Graphics
using CombinatorialSpaces
using Decapodes

using MLStyle
using MultiScaleArrays
using LinearAlgebra
using OrdinaryDiffEq
using SparseArrays
using Statistics
# using BenchmarkTools

import Pkg
Pkg.add("JLD2")
# Pkg.add("GLMakie")
Pkg.add("CairoMakie")
Pkg.add("GeometryBasics")
Pkg.add("FileID")
Pkg.add("MeshIO")
using GeometryBasics: Point2, Point3
using JLD2
# using CairoMakie
using FileIO, MeshIO

Point2D = Point2{Float64}
Point3D = Point3{Float64}

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


LoadError: The following package names could not be resolved:
 * FileID (not found in project, manifest or registry)
[36m   Suggestions:[39m Pro[0m[1mf[22m[0m[1mi[22m[0m[1ml[22m[0m[1me[22mL[0m[1mi[22mkelihoo[0m[1md[22m Hmt[0m[1mF[22macs[0m[1mi[22mmi[0m[1ml[22m[0m[1me[22mBu[0m[1mi[22ml[0m[1md[22mers

In [240]:
Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [241]:
Pkg.add("SyntacticModels")
using SyntacticModels.AMR
using SyntacticModels.ASKEMDecapodes
using SyntacticModels.ASKEMUWDs
using SyntacticModels.Composites

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


## Specify Models

Models can be defined in two ways:
1. SummationDecapode
2. DecaExpr

(1) stores the model as a graph while (2) stores it as an abstract syntax tree.
The latter is preferred for reconstructing the equations of the model.

In [245]:
# Halfar Model

# as a SummationDecapode
halfar_model_sm = @decapode begin
  h::Form0
  Γ::Form1
  n::Constant

  ḣ == ∂ₜ(h)
  ḣ == ∘(⋆, d, ⋆)(Γ * d(h) * avg₀₁(mag(♯(d(h)))^(n-1)) * avg₀₁(h^(n+2)))
end

# Serialize as AMR JSON
open("halfar_model_sm.json", "w") do io 
    d = Dict("header" => AMR.Header("halfar_model", "modelreps.io/SummationDecapode", "Halfar Model", "SummationDecapode", "v1.0"), 
        "model" => generate_json_acset(halfar_model_sm))
    JSON3.pretty(io, d, JSON3.AlignmentContext(indent = 2))
end

In [244]:
# as a DecaExpr
halfar_model_de = ASKEMDecaExpr(
    AMR.Header("halfar_model", "modelreps.io/DecaExpr", "Halfar Model", "DecaExpr", "v1.0"), 
    Decapodes.parse_decapode(quote
      h::Form0
      Γ::Form1
      n::Constant

      ḣ == ∂ₜ(h)
      ḣ == ∘(⋆, d, ⋆)(Γ * d(h) * avg₀₁(mag(♯(d(h)))^(n-1)) * avg₀₁(h^(n+2)))
    end)
)

# Serialize as AMR JSON
open("halfar_model_de.json", "w") do io 
    JSON3.pretty(io, halfar_model_amr, JSON3.AlignmentContext(indent = 2))
end

In [247]:
# Glen's Law Model

# as a SummationDecapode
glen_model_sm = @decapode begin
    Γ::Form1
    A::Constant
    ρ::Constant
    g::Constant
    n::Constant
  
    Γ == (2/(n+2)) * A * (ρ * g)^n
end

# Serialize as AMR JSON
open("glen_model_sm.json", "w") do io 
    d = Dict("header" => AMR.Header("glen_model", "modelreps.io/SummationDecapode", "Glen's Law Model", "SummationDecapode", "v1.0"), 
        "model" => generate_json_acset(glen_model_sm))
    JSON3.pretty(io, d, JSON3.AlignmentContext(indent = 2))
end

# as a DecaExpr
glen_model_de = ASKEMDecaExpr(
    AMR.Header("glen_model", "modelreps.io/DecaExpr", "Glen Model", "DecaExpr", "v1.0"), 
    Decapodes.parse_decapode(quote
        Γ::Form1
        A::Constant
        ρ::Constant
        g::Constant
        n::Constant

        Γ == (2/(n+2)) * A * (ρ * g)^n
    end)
)

# Save to AMR JSON
open("glen_model_de.json", "w") do io 
  JSON3.pretty(io, glen_model_de, JSON3.AlignmentContext(indent = 2))
end

In [96]:
# Dome Model (defined directly)
dome_model = @decapode begin
    h::Form0
    Γ::Form1
    A::Constant
    ρ::Constant
    g::Constant
    n::Constant
    
    # Halfar equation
    ḣ == ∂ₜ(h)
    ḣ == ∘(⋆, d, ⋆)(Γ * d(h) * avg₀₁(mag(♯(d(h)))^(n-1)) * avg₀₁(h^(n+2)))
    
    # Glen's law
    Γ == (2/(n+2)) * A * (ρ * g)^n
end

write_json_acset(dome_model, "dome_model.json")

2256

In [97]:
dome_model_amr = ASKEMDecaExpr(
    AMR.Header("dome_model", "modelreps.io/DecaExpr", "Halfar-Glen Dome Model", "DecaExpr", "v1.0"), 
    Decapodes.parse_decapode(quote
        h::Form0
        Γ::Form1
        A::Constant
        ρ::Constant
        g::Constant
        n::Constant

        # Halfar equation
        ḣ == ∂ₜ(h)
        ḣ == ∘(⋆, d, ⋆)(Γ * d(h) * avg₀₁(mag(♯(d(h)))^(n-1)) * avg₀₁(h^(n+2)))

        # Glen's law
        Γ == (2/(n+2)) * A * (ρ * g)^n
    end)
)

# Save to AMR JSON
open("dome_model_amr.json", "w") do io 
  JSON3.pretty(io, dome_model_amr, JSON3.AlignmentContext(indent = 2))
end

## Compose Models

Several smaller "component" models can be composed together into a "composite" model:
1. Specify a composition (undirected wiring) diagram that connects common variables and parameters together
2. Plug in the component models
3. Optionally flatten the composite model to remove the composition structure

In [249]:
# Specify the composition diagram
dome_model_uwd = @relation () begin
    dynamics(Γ, n)
    stress(Γ, n)
end

# Serialize as JSON
write_json_acset(dome_model_uwd, "dome_model_uwd.json")

298

In [299]:
# Plug in the component models

# using SummationDecapode
dome_model_sm = apex(oapply(dome_model_uwd, [
        Open(halfar_model_sm, [:Γ, :n]), 
        Open(glen_model_sm, [:Γ, :n])
]))

# Save to AMR JSON
open("dome_model_sm.json", "w") do io 
    d = Dict("header" => AMR.Header("dome_model", "modelreps.io/SummationDecapode", "Dome model as flattened composite of Halfar and Glen's law", "SummationDecapode", "v1.0"), 
        "model" => generate_json_acset(dome_model_sm))
    JSON3.pretty(io, d, JSON3.AlignmentContext(indent = 2))
end

In [268]:
# Specify the composition diagram again using UWDExpr
dome_model_uwde = ASKEMUWDs.UWDExpr(
    [Typed(:Γ, :Form1), Typed(:n, :Constant)], 
    [Statement(:dynamics, [Typed(:Γ, :Form1), Typed(:n, :Constant)]), Statement(:stress, [Typed(:Γ, :Form1), Typed(:n, :Constant)])]
)

# Serialize
open("dome_model_uwde.json", "w") do io 
  JSON3.pretty(io, dome_model_uwde, JSON3.AlignmentContext(indent = 2))
end

In [301]:
# Repeat using DecaExpr, UWDExpr, CompositeModelExpr
dome_model_de_comp = CompositeModelExpr(
    AMR.Header("dome_model", "modelreps.io/Composite", "Dome model as composite of Halfar and Glen's law", "CompositeModelExpr", "v0.0"),
    dome_model_uwde,
    [
        OpenModel(halfar_model_de, [:Γ, :n]), 
        OpenModel(glen_model_de, [:Γ, :n])
    ]
)

# Serialize directly as a CompositeModelExpr
open("dome_model_de_comp.json", "w") do io 
    JSON3.pretty(io, dome_model_de_comp, JSON3.AlignmentContext(indent = 2))
end

# Flatten and serialize as a SummationDecapode
open("dome_model_de_flat_sm.json", "w") do io 
    d = Dict("header" => AMR.Header("dome_model", "modelreps.io/SummationDecapode", "Dome model as flattened composite of Halfar and Glen's law", "SummationDecapode", "v1.0"), 
        "model" => generate_json_acset(OpenDecapode(dome_model_de_comp).model.model))
    JSON3.pretty(io, d, JSON3.AlignmentContext(indent = 2))
end

Serializing `CompositeModelExpr` directly appears to generate a non-compliant AMR JSON. Is that correct?

Also, there appears to be two ways to flatten a `CompositeModelExpr`: 
1. `apex(oapply(dome_model_de_comp))`
2. `OpenDecapode(dome_model_de_comp).model.model`

Which is the correct way?

Questions:
1. Is it possible to serialize non-flattened version of the `SummationDecapode` composite model (i.e. the cospan)? 
2. Is it also possible to flatten and then serialize the `CompositeModelExpr` composite model? Currently, `apex(dome_model_de)` throws an error.

## Specify Dimensionality

We need to specify the number of spatial dimensions for which the discrete exterior calculus operators in the model are interpreted.

In [302]:
# 1D
dome_model_1D = expand_operators(dome_model_sm)
infer_types!(dome_model_1D, op1_inf_rules_1D, op2_inf_rules_1D)
resolve_overloads!(dome_model_1D, op1_res_rules_1D, op2_res_rules_1D)

Var,type,name
1,Form0,dynamics_h
2,Form1,Γ
3,Constant,n
4,Form0,dynamics_ḣ
5,infer,dynamics_mult_1
6,Form1,dynamics_mult_2
7,Form1,dynamics_•1
8,infer,dynamics_•2
9,infer,dynamics_•3
10,infer,dynamics_•4


In [303]:
# 2D
dome_model_2D = expand_operators(dome_model_sm)
infer_types!(dome_model_2D)
resolve_overloads!(dome_model_2D)

Var,type,name
1,Form0,dynamics_h
2,Form1,Γ
3,Constant,n
4,Form0,dynamics_ḣ
5,infer,dynamics_mult_1
6,Form1,dynamics_mult_2
7,Form1,dynamics_•1
8,infer,dynamics_•2
9,infer,dynamics_•3
10,infer,dynamics_•4


## Configure Model 

Configure the model by defining a mesh. 
The mesh to be used to discretize the domain and the model needs to be defined:
* using helper functions
* uploading a shapefile
* using custom code

In [106]:
# Define a 1D mesh using a helper function

s_prime_1D = EmbeddedDeltaSet1D{Bool, Point2D}()
add_vertices!(s_prime_1D, 20, point = Point2D.(range(0, 10_000, length = 20), 0))
add_edges!(s_prime_1D, 1:nv(s_prime_1D) - 1, 2:nv(s_prime_1D))
orient!(s_prime_1D)
s_1D = EmbeddedDeltaDualComplex1D{Bool, Float64, Point2D}(s_prime_1D)
subdivide_duals!(s_1D, Circumcenter())

# Save both meshes as shapefile

In [107]:
# Define a 2D rectangular triangulated grid using a helper function

function triangulated_grid(max_x, max_y, dx, dy, point_type)

  s = EmbeddedDeltaSet2D{Bool, point_type}()

  # Place equally-spaced points in a max_x by max_y rectangle.
  coords = point_type == Point3D ? map(x -> point_type(x..., 0), Iterators.product(0:dx:max_x, 0:dy:max_y)) : map(x -> point_type(x...), Iterators.product(0:dx:max_x, 0:dy:max_y))
  # Perturb every other row right by half a dx.
  coords[:, 2:2:end] = map(coords[:, 2:2:end]) do row
    if point_type == Point3D
      row .+ [dx/2, 0, 0]
    else
      row .+ [dx/2, 0]
    end
  end
  # The perturbation moved the right-most points past max_x, so compress along x.
  map!(coords, coords) do coord
    if point_type == Point3D
      diagm([max_x/(max_x+dx/2), 1, 1]) * coord
    else
      diagm([max_x/(max_x+dx/2), 1]) * coord
    end
  end

  add_vertices!(s, length(coords), point = vec(coords))

  nx = length(0:dx:max_x)

  # Matrix that stores indices of points.
  idcs = reshape(eachindex(coords), size(coords))
  # Only grab vertices that will be the bottom-left corner of a subdivided square.
  idcs = idcs[begin:end-1, begin:end-1]
  
  # Subdivide every other row along the opposite diagonal.
  for i in idcs[:, begin+1:2:end]
    glue_sorted_triangle!(s, i, i+nx, i+nx+1)
    glue_sorted_triangle!(s, i, i+1, i+nx+1)
  end
  for i in idcs[:, begin:2:end]
    glue_sorted_triangle!(s, i, i+1, i+nx)
    glue_sorted_triangle!(s, i+1, i+nx, i+nx+1)
  end

  # Orient and return.
  s[:edge_orientation]=true
  orient!(s)
  s
end


s_prime_2D_rect = triangulated_grid(10_000, 10_000, 800, 800, Point3D)
s_2D_rect = EmbeddedDeltaDualComplex2D{Bool, Float64, Point3D}(s_prime_2D_rect)
subdivide_duals!(s_2D_rect, Barycenter())

# Save both meshes as shapefile

In [111]:
# s_prime_2D_rect

In [112]:
# Define a 2D sphere in 3D
s_prime_2D_sph = loadmesh(Icosphere(3, 10_000))
s_2D_sph = EmbeddedDeltaDualComplex2D{Bool, Float64, Point3D}(s_prime_2D_sph)
subdivide_duals!(s_2D_sph, Barycenter())

# Save both meshes as shapefile

In [113]:
# Define a 2D teapot in 3D
download("https://graphics.stanford.edu/courses/cs148-10-summer/as3/code/as3/teapot.obj", "teapot.obj")
s_prime_2D_tea = EmbeddedDeltaSet2D("teapot.obj")
s_2D_tea = EmbeddedDeltaDualComplex2D{Bool,Float64,Point3D}(s_prime_2D_tea)
subdivide_duals!(s_2D_tea, Circumcenter())

# Save both meshes as shapefile

# Parameterize Model

We need to specify values for:
* parameters
* initial conditions
* boundary conditions (optional)

In [115]:
# Parameters
n = 3
ρ = 910
g = 9.8
A = 1e-16

1.0e-16

In [234]:
# Initial conditions

# 1D
h_init_1D = map(point(s_prime_1D)) do (x, _)
        ((7072 - ((x - 5000)^2)) / 9e3 + 2777) / 2777e-1
end

# 2D rectangular triangular grid
h_init_2D_rect = map(point(s_prime_2D_rect)) do (x, y)
  (7072 - ((x - 5000)^2 + (y - 5000)^2)^(1 / 2)) / 9e3 + 10
end

# 2D icosphere in 3D
h_init_2D_sph = map(point(s_prime_2D_sph)) do (x, y, z)
    (z * z) / (10_000 * 10_000)
end

# 2D teapot in 3D
h_init_2D_tea = map(point(s_prime_2D_tea)) do (x, y, z)
    abs(z) * 1.0
end

3644-element Vector{Float64}:
 0.0
 0.08100000023841858
 0.08100000023841858
 0.0
 0.0
 0.0
 0.08100000023841858
 0.08100000023841858
 0.08100000023841858
 0.08100000023841858
 0.14399999380111694
 0.14399999380111694
 0.14399999380111694
 ⋮
 0.06166800111532211
 0.06166800111532211
 0.0
 0.0
 0.05400000140070915
 0.05400000140070915
 0.0579960010945797
 0.0579960010945797
 0.0
 0.0
 0.0
 0.0

In [225]:
extrema(h_init_2D_tea)

(0.0, 3.0)

In [117]:
# Boundary conditions
# None in this example

## Helper Functions

* `generate(...)`

In [127]:
# Implement DEC operators (♯, ♭, ∧, d, ⋆)
function generate(sd, my_symbol; hodge=GeometricHodge())
  op = @match my_symbol begin
    :♯ => x -> begin
      # This is an implementation of the "sharp" operator from the exterior
      # calculus, which takes co-vector fields to vector fields.
      # This could be up-streamed to the CombinatorialSpaces.jl library. (i.e.
      # this operation is not bespoke to this simulation.)
      e_vecs = map(edges(sd)) do e
        point(sd, sd[e, :∂v0]) - point(sd, sd[e, :∂v1])
      end
      neighbors = map(vertices(sd)) do v
        union(incident(sd, v, :∂v0), incident(sd, v, :∂v1))
      end
      n_vecs = map(neighbors) do es
        [e_vecs[e] for e in es]
      end
      map(neighbors, n_vecs) do es, nvs
        sum([nv*norm(nv)*x[e] for (e,nv) in zip(es,nvs)]) / sum(norm.(nvs))
      end
    end
    :mag => x -> begin
      norm.(x)
    end
    :avg₀₁ => x -> begin
      I = Vector{Int64}()
      J = Vector{Int64}()
      V = Vector{Float64}()
      for e in 1:ne(sd)
          append!(J, [sd[e,:∂v0],sd[e,:∂v1]])
          append!(I, [e,e])
          append!(V, [0.5, 0.5])
      end
      avg_mat = sparse(I,J,V)
      avg_mat * x
    end
    :^ => (x,y) -> x .^ y
    :* => (x,y) -> x .* y
    :show => x -> begin
      @show x
      x
    end
    x => error("Unmatched operator $my_symbol")
  end
  return (args...) -> op(args...)
end

generate (generic function with 1 method)

## Generate and Run Simulation

In [230]:
# Sim parameters
start_time = 0.0
end_time = 2e4

# Map constants to model parameters
constants_and_parameters = (
    n = n,
    ρ = ρ,
    g = g,
    A = 1e-16
)

(n = 3, ρ = 910, g = 9.8, A = 1.0e-16)

In [129]:
# 1D Case

# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_1D)], Float64[], [:h])

# Generate simulation
sim = eval(gensim(dome_model_1D, dimension = 1))

# Implement DEC operators on the given mesh
fm = sim(s_1D , generate)

# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
@info("Done")

[ Info: Precompiling Solver
[ Info: Solving


soln.retcode = SciMLBase.ReturnCode.Success


[ Info: Done


In [139]:
# 2D Rectangular Triangulated Grid Case

# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_2D_rect)], Float64[], [:h])

# Generate simulation
sim = eval(gensim(dome_model_2D, dimension = 2))

# Implement DEC operators on the given mesh
fm = sim(s_2D_rect , generate)

# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
@save "2D_rect.jld2" soln
@info("Done")

[ Info: Precompiling Solver
[ Info: Solving


soln.retcode = SciMLBase.ReturnCode.Success


[ Info: Done


In [137]:
# 2D Icosphere in 3D Case

# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_2D_sph)], Float64[], [:h])

# Generate simulation
sim = eval(gensim(dome_model_2D, dimension = 2))

# Implement DEC operators on the given mesh
fm = sim(s_2D_sph , generate)

# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
@save "2D_sph.jld2" soln
@info("Done")

[ Info: Precompiling Solver
[ Info: Solving


soln.retcode = SciMLBase.ReturnCode.Success


[ Info: Done


In [235]:
# 2D Teapot in 3D Case

# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_2D_tea)], Float64[], [:h])

# Generate simulation
sim = eval(gensim(dome_model_2D, dimension = 2))

# Implement DEC operators on the given mesh
fm = sim(s_2D_tea , generate)

(::var"#f#230"{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, var"#121#134"{var"#118#131"}, var"#121#134"{var"#117#130"{EmbeddedDeltaDualComplex2D{Bool, Float64, Point3{Float64}}}}, var"#121#134"{var"#116#129"}, var"#121#134"{var"#109#122"{EmbeddedDeltaDualComplex2D{Bool, Float64, Point3{Float64}}}}, Diagonal{Float64, Vector{Float64}}, SparseMatrixCSC{Int64, Int64}, SparseMatrixCSC{Float64, Int64}, SparseMatrixCSC{Int64, Int64}}) (generic function with 1 method)

In [236]:
# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
@save "2D_teapot.jld2" soln
@info("Done")

[ Info: Precompiling Solver
[ Info: Solving


soln.retcode = SciMLBase.ReturnCode.Success


[ Info: Done


In [237]:
extrema(findnode(soln(start_time), :h))

(0.0, 2.0)

In [238]:
extrema(findnode(soln(end_time), :h))

(-2.908776833042548e-6, 1.9980160202000756)